In [1]:
# Load packages
import os
import pandas as pd
import numpy as np

# This line is needed to display plots inline in Jupyter Notebook
%matplotlib inline

# Required for basic python plotting functionality
import matplotlib.pyplot as plt

# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates
from tqdm import tqdm

# Advanced plotting functionality with seaborn
import seaborn as sns
import folium  #needed for interactive map
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster

sns.set(style="whitegrid")  # can set style depending on how you'd like it to look

## Importing/Exploration

In [3]:
socioeconomic = pd.read_csv('la_socioeconomic_data.csv')
counts = pd.read_csv('LA_NFIP_claims_counts.csv')

In [7]:
print(socioeconomic.columns)
socioeconomic.head(30)

Index(['Unnamed: 0', 'ZIP', 'IncomeBucket1', 'IncomeBucket2', 'IncomeBucket3',
       'IncomeBucket4', 'IncomeBucket5', 'IncomeBucket6', 'IncomeBucket7',
       'IncomeBucket8', 'IncomeBucket9', 'MedianHHIncome', 'MeanHHIncome',
       'PercInsured', 'TotalPop', 'SexRatio', 'MedianAge', 'PercPopUnder18',
       'PercPopOver65', 'PercWhite', 'PercBlack', 'PercAsian', 'PercLatino',
       'HousingUnits', 'UnempRate', 'Pop', 'ZipArea', 'Density',
       'BachelorsRate'],
      dtype='object')


,Unnamed: 0,ZIP,IncomeBucket1,IncomeBucket2,IncomeBucket3,IncomeBucket4,IncomeBucket5,IncomeBucket6,IncomeBucket7,IncomeBucket8,...,PercWhite,PercBlack,PercAsian,PercLatino,HousingUnits,UnempRate,Pop,ZipArea,Density,BachelorsRate
0,0,70001,0.067988,0.052276,0.120379,0.108595,0.137188,0.164048,0.122054,0.132740,...,0.787221,0.131598,0.027832,0.152896,18910,0.043845,37996,5.999,6333.722287,9100.0
1,1,70002,0.063438,0.039505,0.096301,0.080957,0.148861,0.171991,0.107409,0.130196,...,0.756940,0.127771,0.068874,0.170330,9243,0.063429,18879,3.219,5864.864865,6504.0
2,2,70003,0.051750,0.034626,0.099611,0.095157,0.127148,0.206185,0.131539,0.152992,...,0.801894,0.121945,0.041087,0.151738,17085,0.046281,40150,7.019,5720.188061,9036.0
3,3,70005,0.045895,0.032681,0.086064,0.081836,0.157065,0.159708,0.113636,0.152308,...,0.904881,0.027302,0.025437,0.111746,12514,0.040639,24339,4.183,5818.551279,9197.0
4,4,70006,0.058310,0.024534,0.132415,0.103848,0.094606,0.141153,0.126029,0.157621,...,0.794787,0.092450,0.059288,0.226628,6536,0.042713,15703,2.589,6065.276168,4055.0
5,5,70030,0.037262,0.034221,0.062357,0.126996,0.161217,0.167300,0.095817,0.142966,...,0.971610,0.004135,0.000000,0.017916,1533,0.070801,4585,31.498,145.564798,456.0
6,6,70031,0.070565,0.000000,0.114919,0.074597,0.145161,0.167339,0.185484,0.187500,...,0.525622,0.474378,0.000000,0.009517,537,0.051887,1316,3.517,374.182542,130.0
7,7,70032,0.097863,0.046682,0.156355,0.086614,0.136108,0.160292,0.137233,0.134421,...,0.778271,0.119307,0.034659,0.075539,2023,0.088158,3635,1.748,2079.519451,533.0
8,8,70036,0.060606,0.016317,0.139860,0.123543,0.205128,0.263403,0.058275,0.072261,...,0.830798,0.059886,0.000000,0.000000,561,0.035417,1227,4.379,280.200959,90.0
9,9,70037,0.052515,0.040449,0.078178,0.055914,0.152277,0.202923,0.099422,0.191196,...,0.803279,0.098733,0.017884,0.095628,6356,0.046706,15590,62.507,249.412066,2563.0


In [8]:
counts.head(10)

,reportedZipcode,count
0,70000,2
1,70001,5510
2,70002,4450
3,70003,8433
4,70004,4
5,70005,4788
6,70006,4686
7,70009,26
8,70010,3
9,70011,2


In [10]:
merged = socioeconomic.merge(counts, left_on='ZIP', right_on='reportedZipcode', how='left')
merged

,Unnamed: 0,ZIP,IncomeBucket1,IncomeBucket2,IncomeBucket3,IncomeBucket4,IncomeBucket5,IncomeBucket6,IncomeBucket7,IncomeBucket8,...,PercAsian,PercLatino,HousingUnits,UnempRate,Pop,ZipArea,Density,BachelorsRate,reportedZipcode,count
0,0,70001,0.067988,0.052276,0.120379,0.108595,0.137188,0.164048,0.122054,0.132740,...,0.027832,0.152896,18910,0.043845,37996,5.999,6333.722287,9100.0,70001.0,5510.0
1,1,70002,0.063438,0.039505,0.096301,0.080957,0.148861,0.171991,0.107409,0.130196,...,0.068874,0.170330,9243,0.063429,18879,3.219,5864.864865,6504.0,70002.0,4450.0
2,2,70003,0.051750,0.034626,0.099611,0.095157,0.127148,0.206185,0.131539,0.152992,...,0.041087,0.151738,17085,0.046281,40150,7.019,5720.188061,9036.0,70003.0,8433.0
3,3,70005,0.045895,0.032681,0.086064,0.081836,0.157065,0.159708,0.113636,0.152308,...,0.025437,0.111746,12514,0.040639,24339,4.183,5818.551279,9197.0,70005.0,4788.0
4,4,70006,0.058310,0.024534,0.132415,0.103848,0.094606,0.141153,0.126029,0.157621,...,0.059288,0.226628,6536,0.042713,15703,2.589,6065.276168,4055.0,70006.0,4686.0
5,5,70030,0.037262,0.034221,0.062357,0.126996,0.161217,0.167300,0.095817,0.142966,...,0.000000,0.017916,1533,0.070801,4585,31.498,145.564798,456.0,70030.0,76.0
6,6,70031,0.070565,0.000000,0.114919,0.074597,0.145161,0.167339,0.185484,0.187500,...,0.000000,0.009517,537,0.051887,1316,3.517,374.182542,130.0,70031.0,24.0
7,7,70032,0.097863,0.046682,0.156355,0.086614,0.136108,0.160292,0.137233,0.134421,...,0.034659,0.075539,2023,0.088158,3635,1.748,2079.519451,533.0,70032.0,2926.0
8,8,70036,0.060606,0.016317,0.139860,0.123543,0.205128,0.263403,0.058275,0.072261,...,0.000000,0.000000,561,0.035417,1227,4.379,280.200959,90.0,70036.0,252.0
9,9,70037,0.052515,0.040449,0.078178,0.055914,0.152277,0.202923,0.099422,0.191196,...,0.017884,0.095628,6356,0.046706,15590,62.507,249.412066,2563.0,70037.0,1205.0


In [14]:
merged = merged.drop(columns=['reportedZipcode']) #unnecessary
merged['count'] = merged['count'].apply(lambda x : 0 if np.isnan(x) else int(x))
merged

,Unnamed: 0,ZIP,IncomeBucket1,IncomeBucket2,IncomeBucket3,IncomeBucket4,IncomeBucket5,IncomeBucket6,IncomeBucket7,IncomeBucket8,...,PercBlack,PercAsian,PercLatino,HousingUnits,UnempRate,Pop,ZipArea,Density,BachelorsRate,count
0,0,70001,0.067988,0.052276,0.120379,0.108595,0.137188,0.164048,0.122054,0.132740,...,0.131598,0.027832,0.152896,18910,0.043845,37996,5.999,6333.722287,9100.0,5510
1,1,70002,0.063438,0.039505,0.096301,0.080957,0.148861,0.171991,0.107409,0.130196,...,0.127771,0.068874,0.170330,9243,0.063429,18879,3.219,5864.864865,6504.0,4450
2,2,70003,0.051750,0.034626,0.099611,0.095157,0.127148,0.206185,0.131539,0.152992,...,0.121945,0.041087,0.151738,17085,0.046281,40150,7.019,5720.188061,9036.0,8433
3,3,70005,0.045895,0.032681,0.086064,0.081836,0.157065,0.159708,0.113636,0.152308,...,0.027302,0.025437,0.111746,12514,0.040639,24339,4.183,5818.551279,9197.0,4788
4,4,70006,0.058310,0.024534,0.132415,0.103848,0.094606,0.141153,0.126029,0.157621,...,0.092450,0.059288,0.226628,6536,0.042713,15703,2.589,6065.276168,4055.0,4686
5,5,70030,0.037262,0.034221,0.062357,0.126996,0.161217,0.167300,0.095817,0.142966,...,0.004135,0.000000,0.017916,1533,0.070801,4585,31.498,145.564798,456.0,76
6,6,70031,0.070565,0.000000,0.114919,0.074597,0.145161,0.167339,0.185484,0.187500,...,0.474378,0.000000,0.009517,537,0.051887,1316,3.517,374.182542,130.0,24
7,7,70032,0.097863,0.046682,0.156355,0.086614,0.136108,0.160292,0.137233,0.134421,...,0.119307,0.034659,0.075539,2023,0.088158,3635,1.748,2079.519451,533.0,2926
8,8,70036,0.060606,0.016317,0.139860,0.123543,0.205128,0.263403,0.058275,0.072261,...,0.059886,0.000000,0.000000,561,0.035417,1227,4.379,280.200959,90.0,252
9,9,70037,0.052515,0.040449,0.078178,0.055914,0.152277,0.202923,0.099422,0.191196,...,0.098733,0.017884,0.095628,6356,0.046706,15590,62.507,249.412066,2563.0,1205


In [16]:
#export
merged.to_csv('merged_socioeconomic_katrina_data.csv')